<a href="https://colab.research.google.com/github/sourav044/Python/blob/master/image_classification_using_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cat vs. Dog Image Classification

Tasks:-



```
# We will build a keras model to predict dogs and cats.  
#We will be using a pretrained model ResNet18 to predict dogs and cats.
```



#Training and Testing Model 



---

Steps : 


```
# Downloading the files 
# Extracting the files
# Training 
# Testing 


```


















## Explore Dataset

In [0]:
import os

In [2]:
!wget http://files.fast.ai/data/dogscats.zip

--2019-04-28 09:09:03--  http://files.fast.ai/data/dogscats.zip
Resolving files.fast.ai (files.fast.ai)... 67.205.15.147
Connecting to files.fast.ai (files.fast.ai)|67.205.15.147|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 857214334 (818M) [application/zip]
Saving to: ‘dogscats.zip’

dogscats.zip        100%[===================>] 817.50M  40.4MB/s    in 21s     

2019-04-28 09:09:24 (39.5 MB/s) - ‘dogscats.zip’ saved [857214334/857214334]



In [0]:
!unzip dogscats.zip

The contents of the .zip are extracted to the base directory `/content/dogscats/`, which contains `train` and `valid` subdirectories for the training.

In [0]:
train_dir = 'dogscats/train'
validation_dir = 'dogscats/valid'

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

Now, let's see what the filenames look like in the `cats` and `dogs` `train` directories (file naming conventions are the same in the `validation` directory):

In [5]:
train_cat_fnames = os.listdir(train_cats_dir)

print train_cat_fnames[:10]

train_dog_fnames = os.listdir(train_dogs_dir)
train_dog_fnames.sort()
print train_dog_fnames[:10]

['cat.6273.jpg', 'cat.8338.jpg', 'cat.4824.jpg', 'cat.11815.jpg', 'cat.4891.jpg', 'cat.10044.jpg', 'cat.4273.jpg', 'cat.10549.jpg', 'cat.5761.jpg', 'cat.10789.jpg']
['dog.0.jpg', 'dog.1.jpg', 'dog.10.jpg', 'dog.100.jpg', 'dog.1000.jpg', 'dog.10000.jpg', 'dog.10002.jpg', 'dog.10003.jpg', 'dog.10004.jpg', 'dog.10006.jpg']


Let's find out the total number of cat and dog images in the `train` and `validation` directories:

In [6]:
print 'total training cat images:', len(os.listdir(train_cats_dir))
print 'total training dog images:', len(os.listdir(train_dogs_dir))
print 'total validation cat images:', len(os.listdir(validation_cats_dir))
print 'total validation dog images:', len(os.listdir(validation_dogs_dir))

total training cat images: 11500
total training dog images: 11500
total validation cat images: 1000
total validation dog images: 1000


For both cats and dogs, we have 11500 training images and 1000 test images.


## Processing Model  - Training and Testing using Neural Network

The images that will go into our convnet are 150x150 color images (in the next section on Data Preprocessing, we'll add handling to resize all the images to 150x150 before feeding them into the neural network).

Let's code up the architecture. 
We will stack 3 {convolution + relu + maxpooling} modules. 

Our convolutions operate on 3x3 windows and our maxpooling layers operate on 2x2 windows. 
Our first convolution extracts 16 filters, the following one extracts 32 filters, and the last one extracts 64 filters with 128 filter.

In [0]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

In [0]:
model = Sequential()
img_width, img_height = 150,150
#150x150 for the image pixels, and 3 for the three color channels: R, G, and B
img_input = (img_width, img_height, 3)

In [0]:
# First convolution extracts 16 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
model.add(Conv2D(16, (3, 3), input_shape=img_input))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3), input_shape=img_input))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), input_shape=img_input))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# Fourth convolution extracts 128 filters that are 3x3
# Convolution is followed by max-pooling layer with a 2x2 window
#x = layers.Conv2D(128, 3, activation='relu')(x)
#x = layers.MaxPooling2D(2)(x)


In [0]:
# Flatten feature map to a 1-dim tensor so we can add fully connected layers
model.add(Flatten())

# ReLU activation and 512 hidden units
model.add(Dense(512))
model.add(Activation('relu'))


# Add a dropout rate of 0.5
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

Let's summarize the model architecture:

In [47]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 148, 148, 16)      448       
_________________________________________________________________
activation_5 (Activation)    (None, 148, 148, 16)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 72, 72, 32)        4640      
_________________________________________________________________
activation_6 (Activation)    (None, 72, 72, 32)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 36, 36, 32)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 34, 34, 64)        18496     
__________

# Model 


```
We have used keras neural network to create our model and the model has 
```



In [0]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

In [49]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

batch_size_value = 20


# Adding rescale, rotation_range, width_shift_range, height_shift_range,
# shear_range, zoom_range, and horizontal flip to our ImageDataGenerator
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,)

# All images will be rescaled by 1./255
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow training images in batches of 20 using train_datagen generator
train_generator = train_datagen.flow_from_directory(
        train_dir,  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=batch_size_value,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Flow validation images in batches of 20 using test_datagen generator
validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=batch_size_value,
        class_mode='binary')

# class_mode: one of "categorical", "binary", "sparse", "input", "other" or None. Default: "categorical". Mode for yielding the targets:
#"binary": 1D numpy array of binary labels,
#"categorical": 2D numpy array of one-hot encoded labels. Supports multi-label output.

Found 23000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.


In [51]:
model.fit_generator(
      train_generator,
      steps_per_epoch=23000 // batch_size_value,  # 2000 images = batch_size * steps
      epochs=30,
      validation_data=validation_generator,
      validation_steps=2000 // batch_size_value,  # 1000 images = batch_size * steps
      verbose=2)

#Save the model.....
#model.save_weights('first_try.h5')

Epoch 1/30
 - 184s - loss: 0.6232 - acc: 0.6543 - val_loss: 0.5322 - val_acc: 0.7305
Epoch 2/30
 - 185s - loss: 0.5746 - acc: 0.6999 - val_loss: 0.5190 - val_acc: 0.7425
Epoch 3/30
 - 182s - loss: 0.5528 - acc: 0.7175 - val_loss: 0.4461 - val_acc: 0.8030
Epoch 4/30
 - 184s - loss: 0.5223 - acc: 0.7439 - val_loss: 0.4905 - val_acc: 0.7710
Epoch 5/30
 - 183s - loss: 0.5156 - acc: 0.7483 - val_loss: 0.4592 - val_acc: 0.7745
Epoch 6/30
 - 180s - loss: 0.5024 - acc: 0.7563 - val_loss: 0.4652 - val_acc: 0.7775
Epoch 7/30
 - 183s - loss: 0.4821 - acc: 0.7677 - val_loss: 0.4174 - val_acc: 0.8055
Epoch 8/30
 - 182s - loss: 0.4701 - acc: 0.7785 - val_loss: 0.4106 - val_acc: 0.8215
Epoch 9/30
 - 186s - loss: 0.4599 - acc: 0.7860 - val_loss: 0.4092 - val_acc: 0.8145
Epoch 10/30
 - 184s - loss: 0.4480 - acc: 0.7963 - val_loss: 0.3995 - val_acc: 0.8250
Epoch 11/30
 - 181s - loss: 0.4328 - acc: 0.8024 - val_loss: 0.3815 - val_acc: 0.8325
Epoch 12/30
 - 181s - loss: 0.4307 - acc: 0.8038 - val_loss: 0.

#Testing

On top of it we stick two fully-connected layers. Because we are facing a two-class classification problem, i.e. a *binary classification problem*, we will end our network with a [*sigmoid* activation](https://wikipedia.org/wiki/Sigmoid_function), so that the output of our network will be a single scalar between 0 and 1, encoding the probability that the current image is class 1 (as opposed to class 0).

In [53]:
import numpy as np
import random
from tensorflow.keras.preprocessing.image import img_to_array, load_img

# Let's prepare a random input image of a cat or dog from the training set.
cat_img_files = os.path.join(validation_dir,'dogs') + '/dog.10001.jpg'
img_path = cat_img_files

img = load_img(img_path, target_size=(150, 150))  # this is a PIL image
x = img_to_array(img)  # Numpy array with shape (150, 150, 3)
x = x.reshape((1,) + x.shape)  # Numpy array with shape (1, 150, 150, 3)



binary_val = model.predict(x, batch_size=20, verbose=0)
print(binary_val)

[[1.]]
